In [ ]:
%load_ext lab_black

In [ ]:
import altair as alt
import pandas as pd
import sys
import json
import glob
import base64
import numpy as np
from collections import OrderedDict
from operator import getitem
import datetime as dt
from datetime import timedelta
from io import BytesIO
import matplotlib.pyplot as plt
from matplotlib import dates
from matplotlib.dates import WeekdayLocator, DayLocator, DateFormatter, date2num
from matplotlib.dates import MO, TU, WE, TH, FR, SA, SU
import logging
from datetime import datetime
import altair_latimes as lat

alt.themes.register("latimes", lat.theme)
alt.themes.enable("latimes")

### Fetch the ndjson 

In [ ]:
df = pd.read_json(
    "../../DL-COVID-19/DL-us-mobility.ndjson", lines=True, dtype={"fips": str}
)

### Fetch csv format from local copy of Decartes Labs' repo

In [ ]:
src = pd.read_csv(
    "../../DL-COVID-19/DL-us-mobility-daterow.csv",
    dtype={"fips": str, "admin_level": str},
    parse_dates=True,
    infer_datetime_format=True,
).fillna("")

In [ ]:
src["date"] = pd.to_datetime(src["date"])

In [ ]:
src["weekday"] = src["date"].dt.day_name()

---

### Filter the dataframe to California and slim

In [ ]:
ca_state = (
    src[(src["admin1"] == "California") & (src["admin_level"] == "1")]
    .drop(["country_code", "admin_level", "samples"], axis=1)
    .rename(columns={"admin2": "county"})
)

In [ ]:
ca_counties = (
    src[(src["admin1"] == "California") & (src["admin_level"] == "2")]
    .drop(["country_code", "admin_level", "admin1", "samples"], axis=1)
    .rename(columns={"admin2": "county"})
)

In [ ]:
ca_counties["county"] = ca_counties["county"].str.replace(" County", "")

In [ ]:
ca_counties.head()

---

### Filter the dataframe to counties in our region

In [ ]:
socal = [
    "Los Angeles",
    "Ventura",
    "San Bernardino",
    "Riverside",
    "Orange",
]

In [ ]:
socal_counties = ca_counties[ca_counties["county"].isin(socal)]

In [ ]:
socal_counties.head()

---

### Filter the dataframe to Los Angeles County

In [ ]:
la_county = socal_counties[socal_counties["county"] == "Los Angeles"]

In [ ]:
la_county.head()

In [ ]:
alt.Chart(la_county).mark_bar().encode(
    x=alt.X("date:T", axis=alt.Axis(format="%B", title=" ", tickCount=3)),
    y=alt.Y(
        "m50",
        title=" ",
        axis=alt.Axis(format="", title="", tickCount=6),
    ),
    facet=alt.Facet("county"),
).properties(width=1200, height=300, title="m50 in Los Angeles County")

---

### Graph the m50 figure, by county

In [ ]:
alt.Chart(socal_counties).mark_bar().encode(
    x=alt.X("date:T", axis=alt.Axis(format="%b.", title=" ", tickCount=3)),
    y=alt.Y(
        "m50",
        title=" ",
        axis=alt.Axis(format="", title="", tickCount=6),
    ),
    facet=alt.Facet("county"),
).properties(width=300, height=300, title="m50 in SoCal Counties")